# Interactive Brokers Micro Bitcoin Futures Algo V1

In [ ]:
# IBKR_MBT_Algo_V_1.0
'''This strategy will trade when MBT hits its 10 Day Moving Average and Close if the trade turns
against the position taker by $400. This is a work in progress.'''
### Import Libraries
import math
import pandas as pd
import numpy as np
import requests as r
import yfinance as yf
from datetime import datetime, timedelta
datetime.now()

In [ ]:
### must be connected to interactive brokers
from ib_insync import *
util.startLoop() #Use when using jupyter notebooks
ib = IB()
ib.connect('127.0.0.1', 4001, clientId=17) # ibgateway
ib.sleep(2)

In [ ]:
# Qualify Continual Contract for historical prices
con_c = Contract()
con_c.symbol = "MBT"
con_c.secType = "CONTFUT"
con_c.exchange = "CME"
ib.qualifyContracts(con_c)
con_c_ticker = ib.reqMktData(con_c,'',False, False)
con_c_ticker

In [ ]:
# Qualify Contract to trade
con = Contract()
con.symbol = "MBT"
con.secType = "FUT"
con.exchange = "CME"
con.currency = "USD"
con.lastTradeDateOrContractMonth = "202408"
ib.qualifyContracts(con)

In [ ]:
contract = con_c
# Please note that MBT Has been trading since 2022 providing only 3 years of data
bars = ib.reqHistoricalData(contract, endDateTime='',durationStr='5 y',
                            barSizeSetting='1 day', whatToShow='MIDPOINT',
                            useRTH=True, formatDate=1)
bars = pd.DataFrame(bars)
bars.set_index('date', inplace=True)
bars.tail()

In [ ]:
# gather historical data
bars_5yr_high=bars['high'][-1260:-1].max()
bars_5yr_low=bars['low'][-1260:-1].min()
bars_6mo_high=bars['high'][-126:-1].max()
bars_6mo_low=bars['low'][-126:-1].min()
bars_1yr_high=bars['high'][-252:-1].max()
bars_1yr_low=bars['low'][-252:-1].min()
bars_18mo_high=bars['high'][-378:-1].max()
bars_18mo_low=bars['low'][-378:-1].min()
bars_24mo_high=bars['high'][-504:-1].max()
bars_24mo_low=bars['low'][-504:-1].min()
bars_36mo_high=bars['high'][-756:-1].max()
bars_36mo_low=bars['low'][-756:-1].min()
bars_48mo_high=bars['high'][-1008:-1].max()
bars_48mo_low=bars['low'][-1008:-1].min()
bars_h_l=pd.DataFrame({'1 Yr': [bars_1yr_high, bars_1yr_low],'18 Mo':[bars_18mo_high, bars_18mo_low],
                      '2 Yr':[bars_24mo_high, bars_24mo_low],'3 Yr': [bars_36mo_high, bars_36mo_low],
                      '4 Yr':[bars_48mo_high, bars_48mo_low],'5 Yr':[bars_5yr_high,bars_5yr_low]})
bars_h_l.rename(index={0: 'high', 1:'low'},inplace=True)
display(bars_h_l)

In [ ]:
# gather several moving average time fromes
bars_ma=pd.DataFrame(bars['close'])
bars_ma['sma5']=bars_ma.close.rolling(5).mean()
bars_ma['sma10']=bars_ma.close.rolling(10).mean()
bars_ma['sma20']=bars_ma.close.rolling(20).mean()
bars_ma.dropna(inplace=True)
bars_ma=bars_ma.round(2)
display(bars_ma['sma5'].iloc[-1], bars_ma['sma10'].iloc[-1],bars_ma['sma20'].iloc[-1])
bars_ma.tail()

In [ ]:
### bitcoin offer price must be in 5 and 10's
def rtn(n):
    return round(n / 10)* 10
offer = rtn(bars_ma['sma10'].iloc[-1])
offer

In [ ]:
### Calculate risk
btc_risk = (bars_ma['sma10'].iloc[-1]-bars_ma['sma20'].iloc[-1])*.1
btc_risk

In [ ]:
#BTC Buy
con_buy=LimitOrder(action='BUY', totalQuantity=1, lmtPrice=offer, outsideRth=True, tif='GTC')
con_buy_priceCondition = PriceCondition(conId=con.conId,exch='CME', isMore=False, price=offer)
con_buy.conditions.append(con_buy_priceCondition)
con_buy

In [ ]:
#BTC Sell
con_sell=LimitOrder(action='SELL', totalQuantity=1, lmtPrice=offer-4000, outsideRth=True, tif='GTC')
con_sell_priceCondition = PriceCondition(conId=con.conId, exch='CME', isMore=False, price=offer-4000)
con_sell.conditions.append(con_sell_priceCondition)
con_sell

In [ ]:
# place orders
con_buy_order=ib.placeOrder(con, con_buy)
con_sell_order=ib.placeOrder(con, con_sell)

In [ ]:
# check order status
con_buy_order.orderStatus, con_sell_order.orderStatus

In [ ]:
# check current positions
ib.positions()

In [ ]:
# Check open orders
ib.openOrders()

In [ ]:
# cancel all orders
for order in ib.openOrders():
    ib.cancelOrder(order)